# Trumon API

In [1]:
!export CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"

In [2]:
# hanya untuk ujicoba

CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"

# ALTER TABLE Transaksi MODIFY CustomField3 TEXT;

In [3]:
! pip install sqlalchemy

In [4]:
!pip install mysql-connector-python

# Methode to Parser

In [44]:
import re

class Parser:
    
    
    No_mor=None # nomer
    tgl = None # tanggal
    jam_brong=None  # jam dari recipt
    NDP =None # Nilai dan Pajak
    subtotal=None # subtotal belum kena pajak
    pajak=None # Pajak
    tipe=None # take away
    payment=None # tipe payment
    data=None
    RcpSts=None  # Reciept status
    SKU=None 
    data2=None
    def __init__(self, data):
        self.data=data.decode('utf-8','ignore').encode("utf-8").decode('utf-8') # this data
        self.data2=str(data)
        self.getNo_mor()
        self.getPayment()
        self.getNDP()
        self.getTgl()
        self.getSubTotal()
        self.getJam_brong()
        self.getPajak()
        self.getRcpSts()
        self.getTipe()
        self.getSKU()
        
    def getNo_mor(self):
        subRegex = re.compile(r'(Rcpt#:[^\d]+\d+[^\d]+\d+[^\d]+\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.No_mor=mo.group(1)
        else:
            self.No_mor = ""
        
    def getNDP(self): 
        ndpRegex = re.compile(r'[^B]TOTAL[^\d]+(\d+\,\d+)')
        mo = ndpRegex.search(self.data)
        if(mo):
            #print('total: ' + mo.group(1))
            self.NDP=mo.group(1)
        else:
            self.NDP=" "
            # nilai dan pajak disebut sebagai total
        
    def getTgl(self):
        subRegex = re.compile(r'(\d+\/[^\d]+\d{2}\/[^\d]+\d{4})')
        mo = subRegex.search(self.data)
        
        if(re.compile(r'(\d+\/\d+\/\d+)').search(self.data)):
            self.tgl=re.compile(r'(\d+\/\d+\/\d+)').search(self.data).group(1)
        elif(mo):
            self.tgl=mo.group(1)
        
        else:
            self.tgl = ""
            
    def getSubTotal(self):
        subRegex = re.compile(r'SUBTOTAL[^\d]+(\d+\,\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.subtotal=mo.group(1)
        elif(re.compile(r'Closed Bill(\d+,\d+)').search(self.data)):
            self.subtotal=re.compile(r'Closed Bill(\d+,\d+)').search(self.data).group(1)
        else:
            self.subtotal=" "
        
    def getJam_brong(self):
        subRegex = re.compile(r'(\d{2}\:\d{2})')
        mo = subRegex.search(self.data)
        if(mo):
            self.jam_brong=mo.group(1)
        else:
            self.jam_brong=" "
        
    def getPajak(self):
        subRegex = re.compile(r'10%[^\d]+(\d+\,\d+)')
        mo = subRegex.search(self.data)
        if(mo):
            self.pajak=mo.group(1)
        else:
            self.pajak=" "
        
    def getRcpSts(self):
        subRegex = re.compile(r'(TAKE AWAY)')
        mo = subRegex.search(self.data)
        if(re.compile(r'(Sales Day Report)').search(self.data)):
            self.RcpSts=re.compile(r'(Sales Day Report)').search(self.data).group(1)
        elif(re.compile(r'(TAKE AWAY)').search(self.data)):
            self.RcpSts=re.compile(r'(TAKE AWAY)').search(self.data).group(1)
        elif(re.compile(r'(DINE IN)').search(self.data)):
            self.RcpSts=re.compile(r'(DINE IN)').search(self.data).group(1)
        else:
            self.RcpSts=" "
            
    def getTipe(self):
        subRegex = re.compile(r'(TAKE AWAY)')
        mo = subRegex.search(self.data)
        if(re.compile(r'(TAKE AWAY)').search(self.data)):
            self.tipe=re.compile(r'(TAKE AWAY)').search(self.data).group(1)
        elif(re.compile(r'(DINE IN)').search(self.data)):
            self.tipe=re.compile(r'(DINE IN)').search(self.data).group(1)
        else:
            self.tipe=" "
            
    def getPayment(self):
        subRegex = re.compile(r'BNI')
        mo = subRegex.search(self.data)
        if(re.compile(r'(BRI CREDIT CARD)').search(self.data)):
            self.payment=re.compile(r'(BRI CREDIT CARD)').search(self.data).group(1)
        elif(re.compile(r'(GO-PAY)').search(self.data)):
            self.payment=re.compile(r'(GO-PAY)').search(self.data).group(1)
        elif(re.compile(r'(SHOPEE)').search(self.data)):
            self.payment=re.compile(r'(SHOPEE)').search(self.data).group(1)
        elif(re.compile(r'(BCA DEBIT)').search(self.data)):
            self.payment=re.compile(r'(BCA DEBIT)').search(self.data).group(1)
        elif(re.compile(r'(GO-RESTO)').search(self.data)):
            self.payment=re.compile(r'(GO-RESTO)').search(self.data).group(1)
        elif(re.compile(r'(GRABFOOD)').search(self.data)):
            self.payment=re.compile(r'(GRABFOOD)').search(self.data).group(1)
        else:
            self.payment=" "
    
    def getSKU(self):
        # (?<=TAKE AWAY)(.*)(?=SUBTOTAL)
        if(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data)):
            self.SKU=re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data).group(1)
        elif(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data)):
            self.SKU=re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data).group(1)
        elif(re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data2)):
            self.SKU=re.compile(r'(?<=DINE IN)(.*)(?=SUBTOTAL)').search(self.data2).group(1)
        elif(re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data2)):
            self.SKU=re.compile(r'(?<=TAKE AWAY)(.*)(?=SUBTOTAL)').search(self.data2).group(1)
        else:
            self.SKU=""

# Ini bagian service API

In [45]:
#


from fastapi import FastAPI
from starlette.middleware import Middleware
#from fastapi.middleware.cors import CORSMiddleware
from starlette.middleware.cors import CORSMiddleware

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel
import datetime
import os
import sqlalchemy as db
from sqlalchemy import Column, ForeignKey, Integer, String, Numeric, DateTime, ForeignKey, CHAR, Table 

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [46]:
#CONNECTIONSTRING = os.getenv('CONNECTIONSTRING')
CONNECTIONSTRING="mysql+mysqlconnector://pos_almar:@Alm4r2020@localhost/trumon"
engine = create_engine(CONNECTIONSTRING)

Base = declarative_base(engine)

Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

connection = engine.connect()
metadata = db.MetaData()

class Transaksi(Base):
    __tablename__ = "Transaksi"
    __table_args__ = {"autoload": True}
    
class Mentah(Base):
    __tablename__="FileTransferStage2"
    __table_args__ = {"autoload": True}

class DeviceTable(Base):
    __tablename__ = "DeviceTable"
    __table_args__ = {"autoload": True}
    


In [47]:
def getParserMode(devid):
    #
    # untuk mendapatkan mode parser
    #
    getFlag = session.query(DeviceTable).filter(DeviceTable.DeviceId == devid).one()
    return getFlag.ParserMode

# Insert to Transaction

In [48]:
def InsertToTransaction(idFileTransferStage2, DevId, FT, No_mor,
                        tgl, Jam_brong, subtotal, Pjk, NDP, payment, RcpSts, SKU):
    
    # getTrans = session.query(Transaksi).filter(Transaksi.DeviceId == "mantap-mantap").one()
    trans_insert =  Transaksi(RefSN=idFileTransferStage2, DeviceId=DevId, 
                              FileTime=FT, Nomor=No_mor, Tanggal=tgl, Jam=Jam_brong, Nilai=subtotal,
                              Pajak=Pjk, NilaiDanPajak=NDP, CustomField1=payment, CustomField2=RcpSts, CustomField3=SKU)
    session.add(trans_insert)
    session.commit()

# Update Flag
    

In [49]:
#def updateFlag():
def updateFT2Flag(idFTS2, flag):
    FT2_update = session.query(Mentah).filter(Mentah.idFileTransferStage2 == idFTS2).first()
    FT2_update.FlagParser=flag
    session.commit()
    return FT2_update.FlagParser

In [ ]:


middleware = [
    Middleware(CORSMiddleware,
               allow_origins=["*"],
               allow_methods=["*"], # Allows all methods
               allow_headers=["*"], # Allows all headers
              )
]

app = FastAPI(
    middleware=middleware,
    title="Trumon Api",
    description="Just trumon simple api",
    
)
origins = [
    "*",
]


class Makan:
    def simple(self):
        return "Mantap banget"

makan = Makan()


@app.get('/transaksi')
async def GetTrans(DevID: str = "ALMAR0000003"):

    transList = []

    try:
        transaksis = session.query(Transaksi).all()
        
        for item in transaksis:
            #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
            #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
            #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]

            trans = {
                "SeqNum": item.SeqNum,
                "DeviceId": item.DeviceId,
                "RefSN": item.RefSN,
                "FileTime": item.FileTime,
                "Nomor": item.Nomor,
                "Tanggal": item.Tanggal,
                "Jam": item.Jam,
                "Nilai": item.Nilai,
                "Pajak": item.Pajak,
                "NilaiDanPajak": item.NilaiDanPajak,
                "Payment": item.CustomField1,
                "Tipe": item.CustomField2,
                "SKU": item.CustomField3,
                "FlagTransfer": item.FlagTransfer,
            }

            if(item.DeviceId==DevID):
                transList.append(trans)
        #print(transList)
    except:
        session.rollback()
        raise

    return {"data": transList}

@app.get('/parse')
async def letParse(DevID: str = "ALMAR0000003"):
    
    try:
        if(getParserMode(DevID)==3):
            FT2 = session.query(Mentah).filter(db.and_(Mentah.DeviceId == DevID, Mentah.FlagParser ==0)).all()

            count=0
            for item in FT2:
                # parse 
                p1=Parser(item.FileData)
                # insert 
                InsertToTransaction(item.idFileTransferStage2, item.DeviceId, item.FileTime, p1.No_mor,
                                    p1.tgl, p1.jam_brong, p1.subtotal, p1.pajak, p1.NDP, p1.payment, p1.RcpSts, p1.SKU)
                # update Flag
                updateFT2Flag(item.idFileTransferStage2, 1)
                count=count+1

            return {"parsed": count}
        else:
            return {"status": "Mode Parser "+DevID+" bukan 3."}
        #print(transList)
    except:
        session.rollback()
        raise
        
    
@app.get('/parse_reset')
async def delParse(DevID: str = "ALMAR0000003"):
    try:
        # delete transaksi
        delTransaksi = session.query(Transaksi).filter(Transaksi.DeviceId==DevID).delete()
        
        # update flag
        FT2 = session.query(Mentah).filter(db.and_(Mentah.DeviceId == DevID)).all()

        count=0
        for item in FT2:
            
            # update Flag
            updateFT2Flag(item.idFileTransferStage2, 0)
            count=count+1
                
        return {"reset_total": count}
    except:
        session.rollback()
        raise

@app.get('/mentah')
async def getMentah(DevID: str = "ALMAR0000003"):
    dataMentahs = []
    try:
        mentahs = session.query(Mentah).all()
        
        for item in mentahs:
            #contactType = session.query(ContactType).filter(ContactType.ContactTypeId == item.ContactTypeId).one()
            #emails = [item.__dict__ for item in session.query(Email).filter(Email.ContactId == item.ContactId)]
            #phoneNumbers = [item.__dict__ for item in session.query(PhoneNumber).filter(PhoneNumber.ContactId == item.ContactId)]
            
            # convert fileData to string
            stringData = str(item.FileData)
            
            trans = {
                "idFileTransferStage2": item.idFileTransferStage2,
                "data": stringData,
                "filetime": item.FileTime,
                "flagparser": item.FlagParser
            }

            if(item.DeviceId==DevID):
                dataMentahs.append(trans)
        #print(transList)
    except:
        session.rollback()
        raise
    
    return {"data": dataMentahs}


@app.get('/check')
async def checkParser(DevID: str = "ALMAR0000003", RefSN: int=2685):

    transList = []

    try:
        item = session.query(Transaksi).filter(Transaksi.RefSN==RefSN).one()
        mentah = session.query(Mentah).filter(Mentah.idFileTransferStage2==RefSN).one()
        trans = {
            "SeqNum": item.SeqNum,
            "DeviceId": item.DeviceId,
            "RefSN": item.RefSN,
            "FileTime": item.FileTime,
            "Nomor": item.Nomor,
            "Tanggal": item.Tanggal,
            "Jam": item.Jam,
            "Nilai": item.Nilai,
            "Pajak": item.Pajak,
            "NilaiDanPajak": item.NilaiDanPajak,
            "Payment": item.CustomField1,
            "Tipe": item.CustomField2,
            "SKU": item.CustomField3,
            "FlagTransfer": item.FlagTransfer,
            "Data": str(mentah.FileData),
        }

           
        #print(transList)
    except:
        session.rollback()
        raise

    return {"data": trans}
#app.add_middleware(
#    CORSMiddleware,
#    allow_origins=origins,
#    allow_credentials=True,
#    allow_methods=["*"],
#    allow_headers=["*"],
#)

# ngrok tunnel cuma buat akses pake tunnel ngrok 
# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(app, port=8000, host='0.0.0.0', log_level="info", ssl_keyfile="prasimax.net/privkey.pem", ssl_certfile="prasimax.net/cert.pem")
#uvicorn.run(app, port=8000, host='0.0.0.0', log_level="info")

INFO:     Started server process [18563]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on https://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     36.72.213.158:9649 - "GET /parse_reset?DevID=ALMAR0000003 HTTP/1.1" 200 OK
INFO:     36.72.213.158:11531 - "GET /parse?DevID=ALMAR0000003 HTTP/1.1" 200 OK
INFO:     36.72.213.158:31204 - "GET /check?DevID=ALMAR0000003&RefSN=2073 HTTP/1.1" 200 OK
INFO:     36.72.213.158:27862 - "GET /check?DevID=ALMAR0000003&RefSN=2685 HTTP/1.1" 200 OK
INFO:     36.72.213.158:19316 - "GET /check?DevID=ALMAR0000003&RefSN=2073 HTTP/1.1" 200 OK
INFO:     36.72.213.158:9567 - "GET /check?DevID=ALMAR0000003&RefSN=2080 HTTP/1.1" 200 OK
INFO:     36.72.213.158:9089 - "GET /check?DevID=ALMAR0000003&RefSN=2080 HTTP/1.1" 200 OK
INFO:     180.244.173.99:14590 - "GET /check?DevID=ALMAR0000003&RefSN=2080 HTTP/1.1" 200 OK
INFO:     180.244.173.99:18418 - "GET /check?DevID=ALMAR0000003&RefSN=2080 HTTP/1.1" 200 OK
INFO:     36.72.213.158:27858 - "GET /check?DevID=ALMAR0000003&RefSN=2080 HTTP/1.1" 200 OK
INFO:     180.244.173.99:11032 - "GET /check?DevID=ALMAR0000003&RefSN=1855 HTTP/1.1" 200 OK
INFO:     36.72

In [27]:
!ls faoziaziz.id

README	cert.pem  chain.pem  fullchain.pem  privkey.pem
